In [ ]:
import time
import csv
import os
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, StaleElementReferenceException, TimeoutException

In [ ]:
def scrape_hotel_links(url):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = options)
    driver.set_window_size(1120, 1000)
    #url ='https://www.tripadvisor.com/Hotels-g293734-Marrakech_Marrakech_Safi-Hotels.html'
    driver.get(url)
    urls = []

    # give the DOM time to load (3 seconds)
    time.sleep(3)

    # Wait for the element to become clickable
    #wait = WebDriverWait(driver, 10)
    #element = wait.until(EC.element_to_be_clickable((By.XPATH, ".//button[@class='rmyCe _G B- z _S c Wc wSSLS pexOo sOtnj']")))

    # Click the element
    #element.click()

    # Click the "see all" button to reveal the entire hotels.
    #driver.find_element("xpath", ".//button[@class='rmyCe _G B- z _S c Wc wSSLS pexOo sOtnj']").click()

    try:
        collapseDatePanel = driver.find_element("xpath", ".//div[@class='KWdaU Za f e']")
    except:
        collapseDatePanel = None

    if collapseDatePanel is not None:
        collapseDatePanel.click()

    # the number of hotels in Marrakech 778
    numberOfHotels = 30

    while len(urls) < numberOfHotels:

        # give the DOM time to load (3 seconds)
        time.sleep(3)

        # Find all hotels in the current page and store them all to a container
        #container = driver.find_elements("xpath", ".//div[@class='jsTLT K']")
        container = driver.find_elements("xpath", ".//div[contains(@class, 'jsTLT K') or contains(@class, 'listing_title')]/a")

        # Parse each hotel in the container
        for j in range(len(container)):

            print("Progress: {}".format("" + str(len(urls)) + "/" + str(numberOfHotels)))

            if len(urls) >= numberOfHotels:
                break

            # Grab the hotel link
            hotelLink = container[j].get_attribute("href")

            urls.append({
                'Hotel_Link': hotelLink
            })

    # When all pages have been processed, quit the driver
    driver.quit()
    return urls

In [ ]:
# First scrape and save the new links to a DataFrame
links = scrape_hotel_links('https://www.tripadvisor.com/Hotels-g293734-oa30-Marrakech_Marrakech_Safi-Hotels.html')
df = pd.DataFrame(links)
df1 = pd.read_csv('Data/hotels_links.csv')
# Remove the hotel links from df1
df = df[~df['Hotel_Link'].isin(df1['Hotel_Link'])]

# Append the new DataFrame to the existing CSV file
df1 = df1.append(df, ignore_index=True)
df1.to_csv('Data/hotels_links.csv', index=False)
# First save the DataFrame to the CSV file
df.to_csv('Data/hotels_links_30.csv', index=False)

In [ ]:
def scrape_hotel_reviews(url):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = options)
    driver.set_window_size(1120, 1000)
    #url ='https://www.tripadvisor.com/Hotel_Review-g293734-d1649367-Reviews-Marrakech_Ryads_Parc_Spa-Marrakech_Marrakech_Safi.html'
    driver.get(url)
    reviews = []

    # give the DOM time to load (3 seconds)
    time.sleep(3)

    # Grab the hotel name
    hotelName = driver.find_element("xpath","//h1[@id='HEADING']").text

    # Grab the number of english reviews
    numberOfReviewsStr = driver.find_element("xpath",".//label[@class='Qukvo Vm _S']/span[text()='English']/following-sibling::span[1]").text
    numberOfReviews = int(numberOfReviewsStr.strip('()').replace(',', ''))

    while len(reviews) < numberOfReviews:

        # give the DOM time to load (3 seconds)
        time.sleep(3)
        
        # Wait for the element to become clickable
        #wait = WebDriverWait(driver, 10)
        #element = wait.until(EC.element_to_be_clickable((By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]")))

        # Click the element
        #element.click()

        # Click the "expand review" link to reveal the entire review.
        #driver.find_element("xpath", ".//div[contains(@data-test-target, 'expand-review')]").click()
        try:
            driver.find_element("xpath", ".//span[@class='Ignyf _S Z']").click()
        except ElementNotInteractableException:
            element = driver.find_element("xpath", ".//span[@class='Ignyf _S Z']")
            driver.execute_script("arguments[0].click();", element)

        # Find all reviews in the current page and store them all to a container
        container = driver.find_elements("xpath", "//div[contains(@data-test-target, 'HR_CC_CARD')]")

        # Parse each review in the container
        for j in range(len(container)):

            print("Progress: {}".format("" + str(len(reviews)) + "/" + str(numberOfReviews)))

            if len(reviews) >= numberOfReviews:
                break

            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the review date
                    reviewDate = container[j].find_element("xpath", ".//div[@class='cRVSd']/span").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewDate = None
                    break

            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the rating
                    reviewRating = container[j].find_element("xpath", ".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewRating = None
                    break

            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the title
                    reviewTitle = container[j].find_element("xpath",".//div[contains(@data-test-target, 'review-title')]").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewTitle = None
                    break

            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the review text, removing newlines and commas
                    reviewText = container[j].find_element("xpath", ".//q[@class='QewHA H4 _a']").text.replace("\n", "  ").replace(",", "")
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewText = None
                    break
                
            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the reviewer profile link
                    reviewerProfileLink = container[j].find_element("xpath", ".//div[@class='cRVSd']/span/a").get_attribute("href")
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewerProfileLink = None
                    break
                
            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the reviewer Date of stay
                    dateOfStay = container[j].find_element("xpath", ".//span[@class='teHYY _R Me S4 H3']").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    dateOfStay = None
                    break

            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the reviewer Trip type
                    tripType = container[j].find_element("xpath", ".//span[@class='TDKzw _R Me']").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)              
                except NoSuchElementException:
                    tripType = None
                    break

            retry_count = 0
            while retry_count < 3:
                try:
                    # Grab the reviewer location
                    reviewerLocation = container[j].find_element("xpath", ".//span[@class='default LXUOn small']").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    # Handle the case where an element is not found  
                    reviewerLocation = None
                    break

            reviews.append({'Hotel_Name':hotelName,
                            'Review_Date':reviewDate,
                            'Review_Rating':reviewRating,
                            'Review_Title':reviewTitle,
                            'Review_Text':reviewText,
                            'Reviewer_Date_Of_Stay':dateOfStay,
                            'Reviewer_Trip_Type':tripType,
                            'Reviewer_Location':reviewerLocation,
                            'Reviewer_Profile_Link':reviewerProfileLink})

        try:
            # When all the reviews in the container have been processed, move to the next page and repeat
            next_page = driver.find_element("xpath", './/a[contains(@class, "ui_button nav next primary ")]').click()
        except StaleElementReferenceException:
            # If the element has become stale, wait and try again
            time.sleep(2)
            next_page = driver.find_element("xpath", './/a[contains(@class, "ui_button nav next primary ")]').click()
        except NoSuchElementException:
            print('next page failed')

    # When all pages have been processed, quit the driver
    driver.quit()
    return reviews

In [ ]:
hotel_links = pd.read_csv('Data/hotels_links_30.csv')

for index in range(0,len(hotel_links)):
    url = hotel_links['Hotel_Link'][index]
    # First scrape and save the reviews to a DataFrame
    reviews = scrape_hotel_reviews(url)
    df = pd.DataFrame(reviews)

    start_index = url.find("Reviews-") + len("Reviews-")
    end_index = url.find("-Marrakech_Marrakech_Safi")

    hotel_name = url[start_index:end_index]
    # Save the DataFrame to the CSV file
    df.to_csv('Data/Reviews/'+ hotel_name +'.csv', index=False)

In [ ]:
import pandas as pd
import glob

file_list = glob.glob("Data/Reviews/*.csv")

df = pd.DataFrame()

for file_name in file_list:
    temp_df = pd.read_csv(file_name)
    print(len(temp_df))
    df = df.append(temp_df, ignore_index=True)

df.to_csv('merged_reviews.csv', index=False)